In [50]:
import plotly.figure_factory as ff
import numpy as np


In [51]:

bonds_lst = ["IEF", "SHY", "TLT", "SHV", "IEI", "TLH", "BIL", \
    "SPTL", "TMF", "TBF", "VGSH", "VGIT", "VGLT", "SCHO", "SCHR", \
        "SPTS", "GOVT", "TBT", "TMV"]

etfs_lst = ['AMJ', 'IVV', 'VOO', 'VTI', 'QQQ', 'VEA', 'VTV', # 'SPY', 
    'IEFA', 'BND', 'AGG', 'VUG', 'VWO', 'IEMG', 'IJR', 'IJH', 'VIG', 
    'IWF', 'GLD', 'VXUS', 'IWM', 'VO', 'IWD', 'BNDX', # 'WTID'
    'EFA', 'VYM', 'VGT', 'SCHD', 'XLK', 'VCIT', 'VCSH', 'VB', 'ITOT', 
    'XLV', 'BSV', 'XLE', 'LQD', 'VEU', 'RSP', 'MUB', 'VNQ', 'SCHX', 
    'IXUS', 'SCHF', 'XLF', 'IVW', 'IAU', 'USMV', 'DIA', 'IWB', 'IWR', 
    'MBB', 'VTEB', 'VT', 'QUAL', 'VV', 'JPST', 'IGSB', 'EEM', 
    'IVE', 'VBR', 'TIP', 'DGRO', 'IUSB', 'SDY', 'DVY', 'SCHB', 'ACWI', 
    'MDY', 'XLP', 'VGK', 'VHT', 'SCHG', 'VTIP', 'SPYG', 'VMBS', 
    'USFR', 'SPLG', 'EFV', 'VOE', 'SPDW', 'EMB', 'XLU', 'SPYV', 'SCHP']

qqq_lst =  ["MSFT", "AAPL", "AMZN", "NVDA", "GOOGL", "GOOG", "META", "TSLA",         
    "AVGO", "PEP", "COST", "CSCO", "TMUS", "ADBE", "TXN", "CMCSA", "AMD",       
    "NFLX", "QCOM", "HON", "AMGN", "INTC", "INTU", "SBUX", "GILD", "ISRG",      
    "BKNG", "MDLZ", "AMAT", "ADI", "ADP", "REGN", "VRTX", "PYPL", "FISV",       
    "LRCX", "ATVI", "MU", "MELI", "CSX", "CDNS", "PANW", "SNPS", "ORLY",         
    "ASML", "MNST", "MRNA", "MAR", "FTNT", "KLAC", "CHTR", "KDP", # "ABNB",        
    "KHC", "AEP", "DXCM", "CTAS", "LULU", "AZN", "NXPI", "EXC", "MCHP",         
    "ADSK", "BIIB", "PDD", "IDXX", "PAYX", "WDAY", "XEL", "ODFL", "SGEN",       
    "PCAR", "CPRT", "ROST", "ILMN", "EA", "MRVL", "DLTR", # "WBD" "GFS"   
    "FAST", "ENPH", "CTSH", "WBA", "BKR", "VRSK", "CRWD", "ANSS", "CSGP",       
    "ALGN", "FANG", "TEAM", "EBAY", "DDOG", "JD", "ZM", "SIRI", # "CEG",  
    "ZS"] # "LCID", "RIVN"

all_lst = bonds_lst + etfs_lst + qqq_lst
type_index = ['bond'] * len(bonds_lst) + ['etfs'] * len(etfs_lst) \
  + ['stock'] * len(qqq_lst)


In [52]:
ticktype = {}
for i in bonds_lst:
  ticktype[i]='bond'
for i in etfs_lst:
  ticktype[i]='etf'
for i in qqq_lst:
  ticktype[i]='stock'


In [53]:
data_path = '/content/drive/MyDrive/wqu/560/RDA'
import yfinance as yf
import pyarrow 
import pandas as pd
import numpy as np


def dump_price_data(list_tick, start_date, end_date, force=False,):
  import os.path
  import pickle
  res = {}
  pes = {}
  for tick in list_tick:
    # price history
    tick_path = data_path + '/data/raw/' + tick + '_' \
      + start_date + '_' + end_date + '.parquet'
    if not os.path.exists(tick_path) or force:
      tick_data = yf.download(tick,start_date,end_date)
      res[tick]=tick_data
      tick_data.to_parquet(tick_path, engine='pyarrow')
    elif os.path.exists(tick_path):
      res[tick]=pd.read_parquet(tick_path, engine='pyarrow')
  return res

start_date = "2020-04-22"
end_data = "2023-04-22"
individual_tick_data = dump_price_data(all_lst, start_date, end_data, False)


In [54]:
def dump_pe_data(list_tick, force=False,):
  import os.path
  import pickle
  pes = {}
  no_pe = []
  pe_path = data_path + '/data/raw/' + 'pe.pkl'
  if not os.path.exists(pe_path) or force:
    for tick in list_tick:
      # pe info
      try:
        pes[tick]=yf.Ticker(tick).info["trailingPE"]
      except:
        no_pe.append(tick)
      with open(pe_path, 'wb') as f:
        pickle.dump((pes, no_pe), f)
  elif os.path.exists(pe_path):
    with open(pe_path, 'rb') as f:
      (pes, no_pe) = pickle.load(f)
  return pes, no_pe

qqq_pes, no_pe = dump_pe_data(qqq_lst, False)

In [55]:
def mk_high_df(individual_tick_data):
  data = {}
  for k,v in individual_tick_data.items():
    data[k] = v.High
  return pd.DataFrame(data)
  
ticks_high_df = mk_high_df(individual_tick_data)

In [56]:
def mk_daily_returns_df(ticks_high_df):
  mat = np.matrix(ticks_high_df.values)
  return pd.DataFrame(mat[1:,:]/mat[0: -1 ,:] - 1., 
                      index = ticks_high_df.index[1:],
                      columns=ticks_high_df.columns)

In [57]:
dailyreturns_df = mk_daily_returns_df(ticks_high_df)

In [58]:
percent_multiple = 100.
no_days = 252
risk_free_rate = 0.0504 * percent_multiple / no_days

def average_daily_return(hist):
    return percent_multiple * np.mean(hist)

def std(hist):
    return percent_multiple * np.sqrt(no_days) * np.std(hist)

def corrcoef(hist1, hist2):
    from numpy import corrcoef as corrcoefn
    return corrcoefn(hist1, hist2)[0,1]

def skew(hist):
    from scipy.stats import skew as skewn
    return skewn(hist.to_numpy())

def kurtosis(hist):
    from scipy.stats import kurtosis
    return kurtosis(hist.to_numpy())

def covs(hist1, hist2):
    from numpy import cov as covn
    return covn(hist1, hist2)[0,1]



$$
days_{trading} = 252
$$

$$
annual\ risk\ free\ rate = 5.04
$$

$$
Sharpe\ ratio = \sqrt(days_{trading}) \times \frac{100 \times average(returns_{daily}) - \frac{annual\ risk\ free\ rate}{days_{trading}}}{std(100*returns_{daily})}
$$

$$
Sharpe\ ratio = \sqrt(252) \times \frac{100 \times average(returns_{daily}) - \frac{5.04}{252}}{std(100*returns_{daily})}
$$

In [59]:
def sharpe_ratio(hist):
    return np.sqrt(no_days) * (percent_multiple*np.mean(hist) - \
                           (risk_free_rate/no_days))/(np.std(percent_multiple \
                                              * hist))

In [60]:
def mk_sorted_ipes_df(pes):
  ipes = []
  for v in pes.values():
    ipes.append(1. / v)
  ipes_df = pd.DataFrame({"key": list(pes.keys()), 
                         "pe":list(pes.values()),
                         "ipe":ipes})
  ipes_df = ipes_df.sort_values(by='ipe', ascending=False)
  return ipes_df
  

qqq_ipes_sorted_df = mk_sorted_ipes_df(qqq_pes)

In [61]:
def mk_qqq_avg_returns_df(dailyreturns_df):
  returns = {}
  for k in qqq_lst:
    returns[k] = np.mean(dailyreturns_df[k])
  returnsdf = pd.DataFrame({"key":list(returns.keys()), 
                            "return":list(returns.values())})
  returnsdf = returnsdf.sort_values(by=['return'], ascending=False)
  return returnsdf

qqq_returnsdf = mk_qqq_avg_returns_df(dailyreturns_df)

In [62]:
def mk_qqq_std_df(qqq_ipes_sorted_df):
  stds = {}
  for k in qqq_lst:
      stds[k] = std(dailyreturns_df[k])
  stdsdf = pd.DataFrame({"key":list(stds.keys()), "std":list(stds.values())})
  stdsdf['istd'] = 1 / stdsdf['std']
  stdsdf = stdsdf.sort_values(by=['istd'], ascending=False)
  return stdsdf

qqq_stdsdf = mk_qqq_std_df(dailyreturns_df)

In [63]:
def mk_qqq_scoring_df(qqq_returnsdf, qqq_ipes_sorted_df, qqq_stdsdf):
  qqq_evaldf =pd.merge(qqq_ipes_sorted_df, qqq_returnsdf, how="inner")
  qqq_evaldf =pd.merge(qqq_evaldf, qqq_stdsdf, how="inner")
  qqq_evaldf['score'] = (qqq_evaldf['ipe'] / qqq_evaldf['ipe'].max()) + \
    (qqq_evaldf['return']/qqq_evaldf['return'].max()) + \
    (qqq_evaldf['istd']/qqq_evaldf['istd'].max())
  qqq_evaldf = qqq_evaldf.sort_values(by="score", ascending=False)
  return qqq_evaldf

qqq_score_df = mk_qqq_scoring_df(qqq_returnsdf, qqq_ipes_sorted_df, qqq_stdsdf)

In [64]:
qqq_score_df.head(10).style.format().hide(axis="index")


key,pe,ipe,return,std,istd,score
FANG,5.563998,0.179727,0.002406,52.445434,0.019067,2.042006
MRNA,6.506720,0.153687,0.002433,74.928220,0.013346,1.817274
ORLY,26.990643,0.037050,0.001275,20.491775,0.048800,1.355039
PEP,29.367600,0.034051,0.000488,15.417043,0.064863,1.341073
PCAR,12.453152,0.080301,0.000768,24.173766,0.041367,1.323444
ENPH,80.704440,0.012391,0.003216,66.814138,0.014967,1.299688
AVGO,20.803173,0.048070,0.001314,27.006138,0.037029,1.246823
CSX,14.774509,0.067684,0.000697,24.173420,0.041368,1.231134
KLAC,14.688039,0.068083,0.001408,37.311675,0.026801,1.229861
MDLZ,36.903060,0.027098,0.000462,16.652600,0.060051,1.220230


In [65]:
qqq_picks = list(qqq_score_df.iloc[0:1].key)

In [66]:
def mk_rep_stock(tick_list):
  return dailyreturns_df[tick_list].agg(np.mean, axis=1)

rep_stock = mk_rep_stock(qqq_picks)


In [67]:
def find_negcorr_bonds(rep_stock):
  k1 = []
  k2 = []
  corr = []
  for bond in bonds_lst:
    k1.append(bond)
    k2.append(rep_stock)
    corr.append(corrcoef(dailyreturns_df[bond],dailyreturns_df[rep_stock]))
  bondcorrdf = pd.DataFrame({'k1':k1,'k2':k2,'corr':corr})
  bondcorrdf = bondcorrdf.sort_values(by="corr", ascending=True)
  return bondcorrdf

bondcorrdf = find_negcorr_bonds(qqq_picks[0])
bondcorrdf.iloc[0:5].style.format().hide(axis="index")

k1,k2,corr
VGLT,FANG,-0.268296
SPTL,FANG,-0.265670
TLT,FANG,-0.265216
TMF,FANG,-0.264680
TLH,FANG,-0.245801


In [68]:
def find_negcorr_etfs(rep_stock):
  k1 = []
  k2 = []
  corr = []
  for etf in etfs_lst:
    k1.append(etf)
    k2.append(rep_stock)
    corr.append(corrcoef(dailyreturns_df[etf],dailyreturns_df[rep_stock]))
  etfcorrdf = pd.DataFrame({'k1':k1, 'k2':k2, 'corr':corr})
  etfcorrdf = etfcorrdf.sort_values(by="corr", ascending=True)
  return etfcorrdf

etfcorrdf = find_negcorr_etfs(qqq_picks[0])
etfcorrdf.iloc[0:5].style.format().hide(axis="index")

k1,k2,corr
BND,FANG,-0.128119
BNDX,FANG,-0.122599
AGG,FANG,-0.112490
BSV,FANG,-0.083583
IUSB,FANG,-0.081892


In [69]:
def mk_portfolio_daily_returns(dailyreturns_df, stocks, bonds, etfs):
  res = dailyreturns_df[stocks+bonds + etfs]
  res_pretty = res.copy()
  res_pretty.columns = [['stocks'] * len(stocks) +
                   ['bonds']*len(bonds) + ['etfs']*len(etfs),
                  stocks + bonds + etfs]  
  return res, res_pretty


portfolio_daily_returns_df, \
  pretty_portfolio_daily_returns_df = mk_portfolio_daily_returns(
  dailyreturns_df, 
  [qqq_picks[0]],
  bondcorrdf.iloc[0:4].k1.to_list(),
  etfcorrdf.iloc[0:5].k1.to_list())


In [70]:
def protfolio_corr(portfolio_daily_returns_df):
  k1 = []
  k2 = []
  vs = []
  vss = []
  vls = []
  vop = []
  vs_select = []
  top_cols = []
  for k in portfolio_daily_returns_df.columns:
      top_cols.append(ticktype[k]) 
      for j in portfolio_daily_returns_df.columns:
          corr = corrcoef(portfolio_daily_returns_df[k],
                          portfolio_daily_returns_df[j])
          k1.append(k)
          k2.append(j)
          vs.append(corr)
          if k > j:
              vs_select.append(corr)
              if ticktype[k] == 'stock' and ticktype[j] == "stock":
                  vls.append(corr)
              elif ticktype[k] == 'stock' or ticktype[j] == 'stock':
                  vop.append(corr)
              else:
                  vss.append(corr) 
  corrdffinal_df = pd.DataFrame({'k1':k1, 'k2':k2, 'corr':vs})
  portfolio_corr_table = pd.pivot_table(corrdffinal_df, 
                                        values=['corr'], 
                            index=['k1'], columns='k2')

  return corrdffinal_df, portfolio_corr_table, \
    vs, vss, vls, vop, vs_select
  

portfolio_corrdffinal_df, portfolio_corr_table, vs, \
vss, vls, vop, vs_select = protfolio_corr(portfolio_daily_returns_df)


In [71]:
group_labels = ['all'] # name of the dataset
fig = ff.create_distplot([vs_select], group_labels, bin_size=.05)
fig.show()

## Step 1 - For each investment, produce the given statistics:
1. Average return
2. Volatility
3. Skewness
4. Kurtosis

In [72]:
def mk_portfolio_stats(portfolio_daily_returns_df):
  keys =[]
  avgs = []
  stds = []
  skews = []
  kurtoses = []
  sharpe_ratios = []
  dispostion = []

  for k in portfolio_daily_returns_df.columns.tolist():
      keys.append(k)
      avgs.append(no_days * average_daily_return(portfolio_daily_returns_df[k]))
      stds.append(std(portfolio_daily_returns_df[k]))
      skews.append(skew(portfolio_daily_returns_df[k]))
      kurtoses.append(kurtosis(portfolio_daily_returns_df[k]))
      sharpe_ratios.append(sharpe_ratio(portfolio_daily_returns_df[k]))
      dispostion.append(ticktype[k])
  df = pd.DataFrame({'key':keys, "dispostion":dispostion, 
                     "average_returns": avgs, 
                     "volatility": stds, "skew":skews, "kurtosis":kurtoses, 
                     "sharpe_ratio": sharpe_ratios})
  return df

portfolio_stats = mk_portfolio_stats(portfolio_daily_returns_df)

portfolio_stats.style.format().hide(axis="index")

key,dispostion,average_returns,volatility,skew,kurtosis,sharpe_ratio
FANG,stock,60.628965,52.445434,1.089244,7.021818,1.155658
VGLT,bond,-14.567905,14.335664,0.130299,0.973705,-1.017595
SPTL,bond,-14.413764,14.085864,0.182971,0.688050,-1.024698
TLT,bond,-14.889884,14.957370,0.182457,0.707068,-0.996825
TMF,bond,-45.868257,44.571878,0.201550,0.691728,-1.029534
BND,etf,-5.564811,5.278178,0.221771,3.050413,-1.058094
BNDX,etf,-5.033938,5.062035,-0.714793,11.966854,-0.998400
AGG,etf,-5.316244,5.276563,0.346470,3.158213,-1.011311
BSV,etf,-2.602694,2.510813,0.670051,6.660434,-1.044560
IUSB,etf,-4.922566,5.076717,0.237581,3.372176,-0.973575


In [73]:
portfolio_stats[['key', 'volatility']]

,key,volatility
0,FANG,52.445434
1,VGLT,14.335664
2,SPTL,14.085864
3,TLT,14.957370
4,TMF,44.571878
5,BND,5.278178
6,BNDX,5.062035
7,AGG,5.276563
8,BSV,2.510813
9,IUSB,5.076717


In [74]:
def mk_portfolio_cov(portfolio_daily_returns_df):
  k1s = []
  k2s = []
  cov_s = []
  for k1 in portfolio_daily_returns_df.columns.tolist():
    for k2 in portfolio_daily_returns_df.columns.tolist():
      k1s.append(k1)
      k2s.append(k2)
      cov_s.append(covs(portfolio_daily_returns_df[k1], \
                            portfolio_daily_returns_df[k2]))
 
  df = pd.DataFrame({'k1':k1s, "k2": k2s, "cov": cov_s})
  cov_table = pd.pivot_table(df, values=['cov'], 
                              index=['k1'], columns='k2') 
  return cov_table, df

portfolio_cov_table, df = mk_portfolio_cov(portfolio_daily_returns_df)

### Correlation matrix of historical returns.

In [75]:
portfolio_corr_table

corr                                                              \
k2         AGG       BND      BNDX       BSV      FANG      IUSB      SPTL   
k1                                                                           
AGG   1.000000  0.976056  0.762016  0.831526 -0.112490  0.966290  0.846166   
BND   0.976056  1.000000  0.783148  0.833957 -0.128119  0.970022  0.863059   
BNDX  0.762016  0.783148  1.000000  0.693067 -0.122599  0.755931  0.689038   
BSV   0.831526  0.833957  0.693067  1.000000 -0.083583  0.820061  0.601121   
FANG -0.112490 -0.128119 -0.122599 -0.083583  1.000000 -0.081892 -0.265670   
IUSB  0.966290  0.970022  0.755931  0.820061 -0.081892  1.000000  0.821253   
SPTL  0.846166  0.863059  0.689038  0.601121 -0.265670  0.821253  1.000000   
TLT   0.839886  0.856421  0.683284  0.590473 -0.265216  0.814329  0.998504   
TMF   0.837522  0.853755  0.682149  0.586547 -0.264680  0.811693  0.997076   
VGLT  0.836915  0.854750  0.686267  0.602791 -0.268296  0.812174  0.984892   

                                    
k2         TLT       TMF      VGLT  
k1                                  
AGG   0.839886  0.837522  0.836915  
BND   0.856421  0.853755  0.854750  
BNDX  0.683284  0.682149  0.686267  
BSV   0.590473  0.586547  0.602791  
FANG -0.265216 -0.264680 -0.268296  
IUSB  0.814329  0.811693  0.812174  
SPTL  0.998504  0.997076  0.984892  
TLT   1.000000  0.998395  0.984117  
TMF   0.998395  1.000000  0.982832  
VGLT  0.984117  0.982832  1.000000

In [76]:
portfolio_corr_table.loc['FANG']

      k2  
corr  AGG    -0.112490
      BND    -0.128119
      BNDX   -0.122599
      BSV    -0.083583
      FANG    1.000000
      IUSB   -0.081892
      SPTL   -0.265670
      TLT    -0.265216
      TMF    -0.264680
      VGLT   -0.268296
Name: FANG, dtype: float64

### Covariance matrix of historical returns.




In [77]:
portfolio_cov_table

cov                                                              \
k2         AGG       BND      BNDX       BSV      FANG      IUSB      SPTL   
k1                                                                           
AGG   0.000011  0.000011  0.000008  0.000004 -0.000012  0.000010  0.000025   
BND   0.000011  0.000011  0.000008  0.000004 -0.000014  0.000010  0.000025   
BNDX  0.000008  0.000008  0.000010  0.000004 -0.000013  0.000008  0.000020   
BSV   0.000004  0.000004  0.000004  0.000003 -0.000004  0.000004  0.000008   
FANG -0.000012 -0.000014 -0.000013 -0.000004  0.001093 -0.000009 -0.000078   
IUSB  0.000010  0.000010  0.000008  0.000004 -0.000009  0.000010  0.000023   
SPTL  0.000025  0.000025  0.000020  0.000008 -0.000078  0.000023  0.000079   
TLT   0.000026  0.000027  0.000021  0.000009 -0.000083  0.000025  0.000084   
TMF   0.000078  0.000080  0.000061  0.000026 -0.000246  0.000073  0.000249   
VGLT  0.000025  0.000026  0.000020  0.000009 -0.000080  0.000023  0.000079   

                                    
k2         TLT       TMF      VGLT  
k1                                  
AGG   0.000026  0.000078  0.000025  
BND   0.000027  0.000080  0.000026  
BNDX  0.000021  0.000061  0.000020  
BSV   0.000009  0.000026  0.000009  
FANG -0.000083 -0.000246 -0.000080  
IUSB  0.000025  0.000073  0.000023  
SPTL  0.000084  0.000249  0.000079  
TLT   0.000089  0.000264  0.000084  
TMF   0.000264  0.000789  0.000250  
VGLT  0.000084  0.000250  0.000082

In [78]:
def optimize_portfolio(portfolio_daily_returns_df):
  from numpy.random import default_rng
  import random
  random.seed(1111)
  no_items = len(portfolio_daily_returns_df.columns)
  signs_list = []
  weights_list = []
  for i in portfolio_daily_returns_df.columns:
    if ticktype[i] == 'stock':
      signs_list.append(1)
    else:
      signs_list.append(-1)
  signs_series = pd.Series(signs_list)
  return_list = []
  sharpe_list = []
  std_list = []
  portfolio_returns_df_lst = []
  rng = default_rng()
  df = portfolio_daily_returns_df.copy()
  df.columns = pd.core.indexes.range.RangeIndex(0,10,1)
  for i in range(10000):
      weights = list(rng.dirichlet(np.ones(no_items),size=1))
      #weights_list.append(weights)
      w = signs_series*pd.Series(weights[0])
      adjustment = 1 - sum(w)
      w[0] = w[0] + adjustment
      weights_list.append(w)
      portfolio_returns_df = df.dot(pd.Series(w))
      portfolio_returns_df_lst.append(portfolio_returns_df)
      # return_list.append(no_days * np.mean(portfolio_returns_df)) # 
      # return_list.append(no_days * average_daily_return(portfolio_returns_df))
      return_list.append(no_days * percent_multiple * np.mean(portfolio_returns_df))
      std_list.append(std(portfolio_returns_df))
      sharpe_list.append(sharpe_ratio(portfolio_returns_df))
  return weights_list, return_list, sharpe_list, portfolio_returns_df_lst, \
    std_list, signs_list

weights_list, return_list, sharpe_list, portfolio_returns_df_lst, \
std_list, signs_list = optimize_portfolio(portfolio_daily_returns_df)

Max Sharpe Ratio

In [79]:
print("The max Sharpe Ratio is:")
max(sharpe_list)

The max Sharpe Ratio is:


1.3170100530863271

### Portfolio’s average return

In [80]:
print("The return rate for the portfolio is:")
return_list[sharpe_list.index(max(sharpe_list))]

The return rate for the portfolio is:


152.91382010576604

### Portfolio volatility 

In [81]:
print("stdev of portfolio")
std_list[sharpe_list.index(max(sharpe_list))]

stdev of portfolio


116.09161201728823

In [82]:
print("weights for max return")
print()
n=0
for i in portfolio_daily_returns_df.columns:
  print(i, 100 * weights_list[sharpe_list.index(max(sharpe_list))][n]) # signs_list[n] * 
  n += 1


weights for max return

FANG 197.2008583487843
VGLT -0.20449983201954822
SPTL -4.850496212681672
TLT -2.804593363539782
TMF -67.89407250230141
BND -0.23870872710291827
BNDX -7.195711881047308
AGG -7.7110326450300635
BSV -1.3377328527814911
IUSB -4.964010332280125


In [83]:
areturns = []
for i in return_list:
  areturns.append(i)

In [84]:
import plotly.express as px
fig = px.scatter(x=std_list, y=areturns, labels={'x':'volatility', 'y':'% Annual Returns'})
fig.show()

## Step 2 - Answer the following questions:

### Each Team Member/Portfolio Manager will answer the following questions about their own portfolio:

1. Shorting
a. Can this portfolio be sold short? (Hint: Yes, but be sure to explain part b)
b. If it can be sold short, walk through the mechanics.

2. Credit Risk
a. Does this portfolio have credit risk?
b. If it has credit risk, explain what the credit risk is.

3. Portfolio Statistics
a. Compute the weighted return of the portfolio.
b. Compute the variance of the portfolio.

4. Diversification
a. Describe the diversification in words between the two assets.
b. Do you think this portfolio is well diversified or not?

5. Comparing Portfolios
a. How does your portfolio compare to the others in terms of risk?
b. How does your portfolio compare to the others in terms of return?

#### Diversification Design of the Option-c portolio 
The Option-c portfolio adopts the following diversification approaches:
1. It uses three different asset classes. The short postions are all held on bonds or ETFs which exhibit relativly low volitilies due to thier intrinic natures.
2. Most of the portfolio is comprised of assets from relatively stable asset classes, namesly: ETFs and bonds. There is only one stock position.
3. The sole stock positon is shorted by the remainder of the portfolio with constituents that are negatively correlated with the long positon.

### Shorting Mechanism of the Option-c Portfolio
The Option-c portfolio can be sold short. The mechanics to accomplish short sell the portfolio are by taking opposing postions to the portfolio positions -- in otherword we shall short the long positons of the portfolio and instead long the short positions. More specificllay we will to accomplish a short on this portfolio we will take the following postions:

1. FANG -188.356%
2. VGLT 2.622%
3. SPTL 4.323%
4. TLT 2.011%
5. TMF 67.050%
6. BND 6.533%
7. BNDX 0.176%
8. AGG 1.019%
9. BSV 3.417%
10. IUSB 1.192%



### Comments on Credit Risk
The Option-c portfolio has several sources of credit risk. We enumerate the salient ones below:
1. There is only one long postion in the portfolio versus 9 short positons. Although the long positon is dominant representation wise, it is the lone such position. 
2. The short postions are held on investments that have less volatility than the long position -- which provides a questionable hedge. This can be seen below in the long versus short volatilities below:

* long volatility 3.303752
* short volatilities: 0.903, 0.887, 0.942, 2.808, 0.332, 0.319, 0.332,0.158, 0.320

3. The short positions do not have high high inverse correlations with the long positons: -0.112, -0.128, -0.122, -0.084, -0.082, -0.266, -0.265, -0.265, -0.269. None of these correlations approaches a high negative value such as -0.8. The low inverse correlations thus indicate that the short positons provde weak hedges against the sole long positon.

$$
Score = \beta_{volatility} \frac{1}{\left\|volatility\right\|} + \beta_{PE} \frac{1}{\left\|PE\right\|}  + \beta_{returns} \left\| returns \right\|
$$

## Let:
$$
\begin{matrix}
P_{long} && =  && \{FANG \} \\
\\
P_{short} && = && \{ AGG, BND, BNDX, BSV, IUSB, SPTL, TLT, TMF, VGLT \} \\
\\
P && = && P_{long} \cup P_{short} \\
\end{matrix}
$$

## Constraints:
$$
\begin{matrix}
\beta_{i \in P_{long}} && > && 0 \\
\\
\beta_{i \in P_{short}} && < && 0 \\
\\
\Sigma_{i \in P} \beta_i && = && 1
\end{matrix}
$$

## Maximize:
$$
Sharpie\ Ratio(P)
$$
